In [1]:
import pandas as pd
import numpy as np
import configparser
import pymysql
import requests
import re
from tqdm import tqdm
import configparser
import openai
import os
import json

In [2]:
config=configparser.ConfigParser()
config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [3]:
mydb= pymysql.connect(
host=config['iimjobs_db']['hostname'],
user=config['iimjobs_db']['username'],
password=config['iimjobs_db']['password'],
db=config['iimjobs_db']['db_name']
)

mycursor = mydb.cursor()

In [4]:
# mycursor = mydb.cursor()

 
query = "SELECT userid from usr_applied_jobs_info where jobid=1374896"

 # 1372184

mycursor.execute(query)

result = mycursor.fetchall()


In [5]:
udf = pd.DataFrame(result,columns=['userid'])

In [6]:
uid_list = udf.userid.values

In [7]:
uid_list

array([1286631, 2284789, 2499355, ..., 2755388, 2410347, 2358390])

In [8]:
from user_info import *

In [9]:
df = get_user_data_search_embed(uid_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1128/1128 [00:00<00:00, 91931.74it/s]


In [10]:
df.sample(5)

,id,resume,keywords,current_designation,user_experience,professional_info,education_info
187,2542129,Hritvik Bhadane\nCertied Data Analyst &amp; Da...,Analysis Analytics Scientist MySQL BI Data_Ana...,Data Science Intern,2,"[{'id': 3837908, 'userId': None, 'designation'...","[{'id': 97426866, 'userId': 2542129, 'institut..."
61,2710338,Profile\nEntry-level data analyst with a stron...,Analytics Science Data_Visualization Sales Ana...,,0,[],"[{'id': 97619225, 'userId': 2710338, 'institut..."
775,2056875,C O N T A C T\npoorvisrth125@gmail.com\npoorvi...,Legal C Research Corporate Automation R Legal_...,Document specialist,4,"[{'id': 3063384, 'userId': None, 'designation'...","[{'id': 2373145, 'userId': 2056875, 'institute..."
264,1076786,"Bengaluru,India,560017 | 9725636296, | shubham...",Analysis Analyst CFA Automation Quantitative P...,Program Manager,7,"[{'id': 3498621, 'userId': None, 'designation'...","[{'id': 1175447, 'userId': 1076786, 'institute..."
410,2502994,SHUVAM MHAY\nshuvammhay96@gmail.com | 84271709...,Marketing Sales Planning Social_Media Research...,,0,[],"[{'id': 97380594, 'userId': 2502994, 'institut..."


In [11]:
# df[['id','resume']].sample(10).values

In [12]:
# df

In [13]:
df['resume']=df['resume'].str.lower()
df['current_designation']=df['current_designation'].str.lower()

In [14]:
df.sample(5)

,id,resume,keywords,current_designation,user_experience,professional_info,education_info
821,2672520,h\nm\ns\nn\nd\no\nh\ncareer objective\ndedicat...,Analysis R Analytics IT_Recruitment IT Recruit...,recruitment consultant,3,"[{'id': 4038970, 'userId': None, 'designation'...","[{'id': 97572326, 'userId': 2672520, 'institut..."
331,2138527,kuldeep rajpurohit\ni consider myself to be a ...,Analysis Fraudulent Financial Data_Analysis JAVA,citymall,1,"[{'id': 3589272, 'userId': None, 'designation'...","[{'id': 2467695, 'userId': 2138527, 'institute..."
933,2471186,ninad\nsenior analyst\nmumbai +91-7738171477\n...,Analysis SAP Trading Security Regression Autom...,senior analyst,1,"[{'id': 3714069, 'userId': None, 'designation'...","[{'id': 97343630, 'userId': 2471186, 'institut..."
1025,2193039,nikhil yashwanth\nbusiness analyst\nresults-dr...,Analytics ZOHO Supply_Chain CEO MySQL Data_Ana...,business analyst,1,"[{'id': 3286687, 'userId': None, 'designation'...","[{'id': 94672250, 'userId': 2193039, 'institut..."
1055,2724426,professional summary\nexperienced professional...,Sales Analyst Inventory Analysis Onboarding BI...,data analyst,1,"[{'id': 4160507, 'userId': None, 'designation'...","[{'id': 97661155, 'userId': 2724426, 'institut..."


In [15]:
df1= df[['id','current_designation','user_experience','resume']]

In [16]:
# df1['resume']=df1['resume'].str.lower()

In [17]:
df1.sample(5)

,id,current_designation,user_experience,resume
249,2468885,business analyst,2,aakash sunkara\n+919182674563 | aakashvishnu99...
411,2449055,business analyst,3,aakash kathuria\nexperience\nbusiness analyst ...
9,1350180,product manager,3,provided consultation to the client about the...
292,2621254,business analyst intern,1,parneet singh\nanalyst\nabout me\nprofessional...
591,2741564,research analyst,0,academic qualifications\ndegree / examination ...


In [18]:
documents = []
for x in df1:
    y = 'user id: '+df1['id'].values+', '+'user experience: '+df1['user_experience'].values+', '+'user designation: '+df1['current_designation'].values+', '+'user resume: '+df1['resume'].values 
    documents.append(y)

In [19]:
len(documents[0])

1112

In [20]:
documents[0][10]

'user id: 2053027, user experience: 2, user designation: product developer, user resume: preet singh\n+91-870-029-9709 ps8765329@gmail.com www.linkedin.com/in/preetsingh17/ https://github.com/rudopf\nsummary\nelectrical engineering student with software development experience in app development and product development.\nproject experience includes applications of software and hardware. seeking full-time position may 2020 in web devel-\nopment and pharmaceutical production.\neducation\nb.tech, electrical engineering graduating may 2020\nguru gobind singh inderprasth university, delhi, india 3.2/4.0 gpa\nhmr institute of technology\nrelevant coursework: data structures and programmable logic, advanced excel in business,software tools and sys-\ntems programming,probability and computer applications,machine learning and algorithmic game theory.\ntechnical skills\ndata analysis and statistics: python, tableau\ndesign and modeling tools: solidworks, labview, matlab, microsoft office\nprogram

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.vectorstores import FAISS


In [22]:
# documents = df1.values

In [23]:
# documents[90]

In [24]:
# documents[:, 0] = [{'id':int(value)} for value in documents[:, 0]]


In [25]:
# documents[0][90]

In [26]:
# len(documents)

In [36]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000,chunk_overlap=0)
splits = text_splitter.split_text(str(documents[0]))
print(len(splits), splits[0])

8 ['user id: 1286631, user experience: 5, user designation: product manager, user resume: a r a v i n d s u b r a m a n i a n g\n5e4, happy windows, kattupakkam, chennai-600056 a.sbmn1994@gmail.com (+91)-9751053481\neducational qualifications\ncourse name year of\ncompletion\ninstitute name aggregate\nb.tech (production\nengineering)\n2015 national institute of technology tiruchirappalli 7.97\n(cgpa)\nclass xii (cbse) 2011 jawahar senior secondary school, neyveli 85.6%\nclass x (cbse) 2009 jawahar higher secondary school, neyveli 93.4%\nprofessional experience\nrbl bank private limited, manager cards portfolio, gurugram may 2022 apr 2023\n managing p&amp;l ownership of the emi products of credit cards x-sell portfolio\n monitoring and improving key metrics of the business like penetration, conversion and income\n implemented digital transformation and increased the topline and bottomline of business\nindustrybuying, category owner, new delhi jun 2021 jul 2021\n managed seller on-boardi

In [37]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cuda'})

In [38]:
db = FAISS.from_texts(documents[0], embeddings)

In [39]:
import langchain
from queue import Queue
from typing import Any
from langchain.llms.huggingface_text_gen_inference import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import LLMResult
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts.prompt import PromptTemplate
from anyio.from_thread import start_blocking_portal #For model callback streaming
import re
import gradio as gr
langchain.debug=True 

In [40]:
model = 'meta-llama/Llama-2-7b-chat-hf'

In [41]:
from bs4 import BeautifulSoup

In [42]:
text = '''
<p><b>About the company:</b><br/><br/>Bizup is an early stage startup, driven by the vision to empower the 60 Mn small businesses (SMEs) in India with technology. We are building the first of its kind short video app for B2B (business to business) transactions - think of it like Tiktok for businesses. Aim is to build India's largest network of business users, with the plan to onboard 2 Mn+ users in next 12 months, largely from Tier 2+ cities. Founders are IIT/IIM/ISB alumni, with leadership experience at McKinsey, Paytm, Tata (linkedin.com/in/suparngoel, linkedin.com/in/anshul-singhal-21696810/). If you are someone looking to join in the 0 to 1 journey of building a rocketship, in a huge opportunity area, then reach out to us!<br/><br/><b>About the role:</b><br/><br/>- In the dynamic position of Business Analyst, you will play a pivotal role in driving user engagement and satisfaction for our app. Your responsibilities will encompass a diverse range of critical functions, making this role both exciting and impactful. Here's what you will be doing:<br/><br/>- Analyze user data to understand behaviour, preferences, and trends, using these insights to shape customer engagement strategies<br/><br/>- Monitor day-to-day operations and manage the team to ensure smooth functioning in compliance with company standards<br/><br/>- Collaborate closely with cross-functional teams, including business, product development, and content team, to ensure seamless communication and alignment in delivering a unified user experience<br/><br/>- Maintain user engagement dashboards and prepare reports and present findings to senior management.<br/><br/><b>What we are looking for?</b><br/><br/>- Any graduate, or similar<br/><br/>- Strong analytical skills and proficiency in using software and tools for data analysis like Excel<br/><br/>- Familiarity with communication platforms, CRMs, and a willingness to stay updated on emerging tech trends<br/><br/>- Proven experience in managing relationships across cross functional teams and team management or a similar role</p>
'''

# Use BeautifulSoup to remove HTML tags
soup = BeautifulSoup(text, "html.parser")
cleaned_text = soup.get_text()

print(cleaned_text)


About the company:Bizup is an early stage startup, driven by the vision to empower the 60 Mn small businesses (SMEs) in India with technology. We are building the first of its kind short video app for B2B (business to business) transactions - think of it like Tiktok for businesses. Aim is to build India's largest network of business users, with the plan to onboard 2 Mn+ users in next 12 months, largely from Tier 2+ cities. Founders are IIT/IIM/ISB alumni, with leadership experience at McKinsey, Paytm, Tata (linkedin.com/in/suparngoel, linkedin.com/in/anshul-singhal-21696810/). If you are someone looking to join in the 0 to 1 journey of building a rocketship, in a huge opportunity area, then reach out to us!About the role:- In the dynamic position of Business Analyst, you will play a pivotal role in driving user engagement and satisfaction for our app. Your responsibilities will encompass a diverse range of critical functions, making this role both exciting and impactful. Here's what y

In [43]:
from langchain.llms import VLLM

llm = VLLM(model=model,
           trust_remote_code=True,  
           max_new_tokens=1024,
           top_k=20,
           top_p=0.95,
           temperature=0)

INFO 02-14 15:18:44 llm_engine.py:73] Initializing an LLM engine with config: model='meta-llama/Llama-2-7b-chat-hf', tokenizer='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)
INFO 02-14 15:18:50 llm_engine.py:223] # GPU blocks: 805, # CPU blocks: 512
INFO 02-14 15:18:53 model_runner.py:394] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-14 15:19:02 model_runner.py:437] Graph capturing finished in 9 secs.


In [44]:
template = """
<s>[INST] You are an AI agent working for a job platform and your task is to recommend the most relevant users 
using the context (in the form of resume text provided. If no context provided, answer like a AI assistant.
{context}
Question: {question} [/INST]
"""

retriever = db.as_retriever(
        search_kwargs={"k": 2}
    )

In [45]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever,     
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    }
)

In [46]:
result = qa_chain({"query": "give me ids of users who have experience with BOTH tableau and power BI?"})
# print(result)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "give me ids of users who have experience with BOTH tableau and power BI?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "give me ids of users who have experience with BOTH tableau and power BI?",
  "context": "user id: 2122480, user experience: 2, user designation: data analyst, user resume: sonakshi jaiswal\n(+91) 8090186665, 9569296787\n@ sonakshijaiswal444@gmail.com\n# www.linkedin.com/in/sonakshi-jaiswal\n# https://github.com/sonakshi-jaiswal\nsummary\n results-driven data analyst with 2 years of experience in machine learning, data analysis, and\nvisualization. skilled in utilizing tools such as power bi, tableau, python, and sql to derive\nactionable insights and drive business growth.\n demonstrated ability t

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.05s/it]

[llm/end] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain > 5:llm:VLLM] [4.07s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Based on the provided resumes, the following are the IDs of users who have experience with both Tableau and Power BI:\n\n1. Sonakshi Jaiswal - user ID: 2122480\n2. Rohit Batthula - user ID: 2723804\n\nBoth Sonakshi and Rohit have experience working with Tableau and Power BI, as mentioned in their resumes.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] [4.07s] Exiting Chain run with output:
{
  "text": "Based on the provided resumes, the following are the IDs of users who have experience with both Tableau and Power BI:\n\n1. Sonakshi Jaiswal - user ID: 2122480\n2. Rohit Batthula - user ID: 2723804\n\nBoth Sonakshi and Rohit have experience working with 

In [76]:
pattern = re.compile(r'\d+\.\s*([^\n]+)\s*-\s*(\d+)')

# Find all matches in the text
matches = pattern.findall(result['result'])

# Extract User IDs and print them
user_ids = [(name.strip(), id) for name, id in matches]

In [77]:
user_ids

[]

In [78]:
uid_list = []
for user_id in user_ids:
    print(f"User: {user_id[0]}, ID: {user_id[1]}")
    uid_list.append(user_id[1])

In [79]:
uid_list

[]

In [80]:
# user_id_pattern = r'User ID: (\d+)' and r'id: (\d+)' and r'ID: (\d+)' and r'ID is : (\d+)' and r"His ID is (\d+)" and r'ID: (\d+)' and r'Id: (\d+)' and r' ID: (\d+)'and r' id: (\d+)'and r'ID: (\d+)' and r' id: (\d+)'
# user_id_match = re.search(r'User ID: (\d+)', result['result'])
# user_id = user_id_match.group(1)

user_ids = re.findall(r'User ID: (\d+)' and r'ID (\d+)' and r'user id: (\d+)', result['result'])

# user_ids = re.findall(user_id_pattern, result['result'])

In [81]:
user_ids

[]

In [47]:
data = get_user_data_search_embed(user_ids)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 27060.03it/s]


In [48]:
data

,id,resume,keywords,current_designation,user_experience,professional_info,education_info
0,2553080,SRIJEN JHINGTA Email: srijenj@gmail.com Ph.no....,Research Retail Client Brand Analysis Marketin...,Consultant - Business Consulting PI,2,"[{'id': 4252712, 'userId': None, 'designation'...","[{'id': 97439491, 'userId': 2553080, 'institut..."
1,2553080,SRIJEN JHINGTA Email: srijenj@gmail.com Ph.no....,Research Retail Client Brand Analysis Marketin...,Consultant - Business Consulting PI,2,"[{'id': 4252712, 'userId': None, 'designation'...","[{'id': 97439491, 'userId': 2553080, 'institut..."


In [198]:
df1.loc[df1['id']=='2312967']

,id,current_designation,user_experience,resume


In [199]:
df1.loc[df1['id']=='4567890']

,id,current_designation,user_experience,resume


In [200]:
# data['resume'].values

In [86]:
# user_id_pattern

In [88]:
# df.shape

In [89]:
# df.loc[df['resume'].str.contains('gaming')]['resume'].values

In [ ]:
# give me all those user ids atleat 4 who have worked as analyst
# give all those user with their id who have worked as business development manager